In [ ]:
!pip install gradio

In [ ]:
import pickle
import re
from difflib import get_close_matches
import gradio as gr
from transformers import pipeline

In [ ]:
# Load saved data
movies = pickle.load(open("movie_list.pkl", "rb"))
similarity = pickle.load(open("similarity.pkl", "rb"))

In [ ]:
# Load Hugging Face conversational model
chatbot = pipeline("text2text-generation", model="google/flan-t5-base")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
def recommend(movie_name, num_recommendations=5):
    try:
        # Try to find the closest match in the database
        movie_name = movie_name.lower().strip()

        # Check for exact match first
        exact_match = movies[movies['title'].str.lower() == movie_name]
        if not exact_match.empty:
            index = exact_match.index[0]
        else:
            # Try fuzzy matching - look for movies containing the search term
            contains_match = movies[movies['title'].str.lower().str.contains(movie_name)]
            if not contains_match.empty:
                # If multiple matches, take the most popular one (could be improved)
                index = contains_match.index[0]
            else:
                # Try to find the closest match by string similarity
                matches = get_close_matches(movie_name, movies['title'].str.lower(), n=1, cutoff=0.6)
                if matches:
                    index = movies[movies['title'].str.lower() == matches[0]].index[0]
                else:
                    return None, None  # No match found

        distances = similarity[index]
        movie_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:num_recommendations+1]
        recommendations = [movies.iloc[i[0]].title for i in movie_list]
        original_title = movies.iloc[index].title
        return original_title, recommendations
    except Exception as e:
        print(f"Error in recommend function: {e}")
        return None, None


In [ ]:
def parse_input(user_input):
    try:
        # Check for patterns like "recommend 10 movies like Harry Potter"
        pattern = r'recommend (\d+) movies? like (.+)'
        match = re.search(pattern, user_input, re.IGNORECASE)

        if match:
            num_rec = int(match.group(1))
            movie_name = match.group(2).strip()
            return movie_name, min(num_rec, 20)  # Limit to 20 recommendations max

        # Check alternative patterns
        alt_pattern = r'movies? similar to (.+)'
        alt_match = re.search(alt_pattern, user_input, re.IGNORECASE)
        if alt_match:
            return alt_match.group(1).strip(), 5

        # Default case - assume the entire input is a movie title
        return user_input.strip(), 5
    except Exception as e:
        print(f"Error in parse_input: {e}")
        return user_input.strip(), 5

In [ ]:
def respond(message, chat_history):
    try:
        # Parse the input to get movie name and number of recommendations
        movie_name, num_rec = parse_input(message)

        # Get recommendations
        original_title, recommendations = recommend(movie_name, num_rec)

        if recommendations is None:
            response = f"I couldn't find '{movie_name}' in my database. Could you try a different movie title?"
            chat_history.append((message, response))
            return "", chat_history

        # Format recommendations
        if num_rec == 1:
            rec_text = f"Here's 1 movie similar to {original_title}:\n"
        else:
            rec_text = f"Here are {num_rec} movies similar to {original_title}:\n"

        rec_text += "\n".join(f"{i+1}. {title}" for i, title in enumerate(recommendations))

        # Generate conversational response
        prompt = f"""User: I like the movie {original_title}. Recommend something similar.
Bot: I'd be happy to recommend some movies similar to {original_title}! """

        try:
            response = chatbot(prompt, max_length=150)[0]['generated_text']
        except:
            response = f"I think you'll enjoy these movies similar to {original_title}:"

        # Combine both responses
        full_response = f"{response}\n\n{rec_text}"
        chat_history.append((message, full_response))
        return "", chat_history
    except Exception as e:
        print(f"Error in respond function: {e}")
        error_response = "Sorry, I encountered an error processing your request. Please try again."
        chat_history.append((message, error_response))
        return "", chat_history



In [ ]:
css = """
/* Fix user messages */
.gr-chat-message.gr-chat-message-user .gr-chat-message-text {
    color: #000000 !important;
    opacity: 1 !important;
    font-weight: 600;
}

/* Fix bot messages */
.gr-chat-message.gr-chat-message-bot .gr-chat-message-text,
.gr-chat-message.gr-chat-message-bot .markdown {
    color: #1e1e1e !important;
    opacity: 1 !important;
    font-weight: 500;
}

/* Optional: improve markdown elements inside bot responses */
.gr-chat-message-bot .markdown p,
.gr-chat-message-bot .markdown li {
    color: #1e1e1e !important;
    opacity: 1 !important;
    font-weight: 500;
}
"""
with gr.Blocks(theme=gr.themes.Base(), css=css) as demo:
    gr.Markdown("""
    # 🎬 Movie Recommender Chatbot
    ### Powered by Machine Learning & Natural Language Processing
    """)

    with gr.Row():
        with gr.Column(scale=3):
            chatbot_ui = gr.Chatbot(height=500, label="Chat History")
            msg = gr.Textbox(
                placeholder="Type a movie you like (e.g., 'Harry Potter') or ask for recommendations...",
                label="Your Message"
            )
        with gr.Column(scale=1):
            gr.Markdown("### Examples to try:")
            gr.Examples(
                examples=[
                    "The Dark Knight",
                    "recommend 3 movies like Inception",
                    "movies similar to Titanic",
                    "Harry Potter",
                    "The Shawshank Redemption"
                ],
                inputs=msg,
                label="Try these examples"
            )

    with gr.Row():
        clear = gr.Button("Clear Chat", variant="secondary")
        submit = gr.Button("Send", variant="primary")

    msg.submit(respond, [msg, chatbot_ui], [msg, chatbot_ui])
    submit.click(respond, [msg, chatbot_ui], [msg, chatbot_ui])
    clear.click(lambda: None, None, chatbot_ui, queue=False)

<ipython-input-12-428ab541d03a>:33: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot_ui = gr.Chatbot(height=500, label="Chat History")


In [ ]:
demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c97e8331641b33439e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
